# Trained Model Visualization

In [2]:
from utils.utils import *

**Important**: If you get the error that references the module `SRC.models.models`, then run the following lines of code to add the explicit path to the folder SRC.

In [1]:
# Only run if necessary

import sys
full_path_to_SRC = 'C:/Users/donai/Documents/MyDocs/Universitat/Tercer curs/Semestre 2/Xarxes Neuronals i Aprenentatge Profund/Exercicis/xnap-project-matcad_grup_10'
sys.path.append(full_path_to_SRC)


## Loading the necessary data

**Important**: Change the following paths to match the location where your data is stored.

In [3]:
DATA_LOCATION = '../data'
model_path = DATA_LOCATION + '/logs/EncoderDecorder_model.pth'
device = 'cpu' # or 'cuda'

In [4]:
model = load_ED_model(model_path, device)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to C:\Users\donai/.cache\torch\hub\checkpoints\resnet50-11ad3fa6.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [5]:
model

EncoderDecoder(
  (encoder): EncoderCNN(
    (resnet): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
 